In [1]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [2]:
# | hide
import sys,os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path.cwd().parent))
# os.chdir(Path.cwd().parent / 'extracao')


In [3]:
# | export
import re
from typing import Iterable, Tuple, Union, List

import pandas as pd
import numpy as np
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from rich.progress import Progress
from pyarrow import ArrowInvalid
from dotenv import load_dotenv, find_dotenv


from extracao.constants import BW, BW_pattern, APP_ANALISE_PT, APP_ANALISE_EN

RE_BW = re.compile(BW_pattern)
MAX_DIST = 10  # Km
LIMIT_FREQ = 84812.50
load_dotenv(find_dotenv(), override=True)

True

In [4]:
# | export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}") from e
    return df


# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [5]:
# | export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:    # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match[2]]
        if mantissa := match[3]:
            number = float(f"{match[1]}.{mantissa}")
        else:
            number = float(match[1])
        classe = match[4]
        return str(multiplier * number), str(classe)
    return "-1", "-1"


In [6]:
# | export
def _filter_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Recebe a base de dados da Anatel e formata as colunas para leitura de acordo com os requisitos do Matlab"""
    df["#Estação"] = df["Número_Estação"]
    df.loc[df.Multiplicidade != "1", "#Estação"] = (
        df.loc[df.Multiplicidade != "1", "Número_Estação"]
        + "+"
        + df.loc[df.Multiplicidade != "1", "Multiplicidade"]
    )
    cols_desc = [
        "Fonte",
        "Status",
        "Classe",
        "Entidade",
        "Fistel",
        "#Estação",
        "Município_IBGE",
        "UF",
    ]
    df.loc[:, cols_desc].fillna("NI", inplace=True)

    df["Descrição"] = (
        "["
        + df.Fonte
        + "] "
        + df.Status
        + ", "
        + df.Classe
        + ", "
        + df.Entidade.str.title()
        + " ("
        + df.Fistel
        + ", "
        + df["#Estação"]
        + "), "
        + df.Município_IBGE
        + "/"
        + df.UF
    )

    bad_coords = df.Coords_Valida_IBGE == "0"

    df.loc[bad_coords, "Descrição"] = df.loc[bad_coords, "Descrição"] + "*"

    df.loc[bad_coords, ["Latitude", "Longitude"]] = df.loc[
        bad_coords, ["Latitude_IBGE", "Longitude_IBGE"]
    ].values

    df = df.loc[:, APP_ANALISE_PT]
    df.columns = APP_ANALISE_EN
    return df

In [7]:
# | export
def _format_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Formata o arquivo final de dados para o formato esperado pela aplicação em Matlab"""
    df = df.astype("string")
    df.loc[len(df), :] = [
        "-1",
        "-15.7801",
        "-47.9292",
        "[TEMP] L, FX, Estação do SMP licenciada (cadastro temporário)",
        "10",
        "999999999",
        "NI",
        "-1",
    ]  # Paliativo...
    for c in ["Latitude", "Longitude"]:
        df[c] = df[c].fillna(-1).astype("float32")
    df["Frequency"] = df["Frequency"].astype("float64")
    df.loc[df.Service.isin(["", "-1"]), "Service"] = pd.NA
    df["Service"] = df.Service.fillna("-1").astype("int16")
    df.loc[df.Station.isin(["", "-1"]), "Station"] = pd.NA
    df["Station"] = df.Station.fillna("-1").astype("int32")
    df.loc[df.BW.isin(["", "-1"]), "BW"] = pd.NA
    df["BW"] = df["BW"].astype("float32").fillna(-1)
    df.loc[df["Class"].isin(["", "-1"]), "Class"] = pd.NA
    df["Class"] = df.Class.fillna("NI").astype("category")
    df = df[df.Frequency <= LIMIT_FREQ]
    df.sort_values(
        by=["Frequency", "Latitude", "Longitude", "Description"], inplace=True
    )
    unique_columns = df.columns.tolist()
    unique_columns.remove('Description')
    df = df.drop_duplicates(subset=unique_columns, keep="last").reset_index(drop=True)
    df["Id"] = [f"#{i+1}" for i in df.index]
    df["Id"] = df.Id.astype("string")
    df.loc[df.Description == "", "Description"] = pd.NA
    df["Description"] = df["Description"].astype("string").fillna("NI")
    return df[["Id"] + list(APP_ANALISE_EN)]

### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [8]:
# | export
def get_km_distance(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km


def merge_on_frequency(
    df_left: pd.DataFrame,  # DataFrame da esquerda a ser mesclado
    df_right: pd.DataFrame,  # DataFrame da direira a ser mesclado
    on: str = "Frequency",  # Coluna usada como chave de mesclagem
    coords: Tuple[str] = ("Latitude", "Longitude"),
    description: str = "Description",
    suffixes: Tuple[str] = ("_x", "_y"),  # Sufixo para as colunas que foram criadas
) -> pd.DataFrame:  # DataFrame resultante da mesclagem
    """Mescla os dataframes baseados na frequência
    É assumido que as colunas de ambos são idênticas, caso contrário os filtros não irão funcionar como esperado
    """
    df: pd.DataFrame = pd.merge(
        df_left.astype("string"),
        df_right.astype("string"),
        on=on,
        how="outer",
        suffixes=suffixes,
        indicator=True,
        copy=False,
    )

    x, y = suffixes
    lat, long = coords


    left = df._merge == "left_only"
    right = df._merge == "right_only"
    both = df._merge == "both"
    df = df.drop(columns=["_merge"])
    
    left_cols: List[str] = [c for c in df.columns if y not in c]
    right_cols: List[str] = [c for c in df.columns if x not in c]


    only_left = df.loc[left, left_cols].drop_duplicates(ignore_index=True)
    only_left.columns = [c.removesuffix(x) for c in left_cols]

    only_right = df.loc[right, right_cols].drop_duplicates(ignore_index=True)
    only_right.columns = [c.removesuffix(y) for c in right_cols]

    both_columns = [f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]

    df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

    close = df.loc[both, "Distance"] <= MAX_DIST
    
    df_close = df.loc[(both & close)].drop_duplicates(ignore_index=True)
    
    df_close[f"{description}{x}"] = (
        df_close[f"{description}{x}"] + " | " + df_close[f"{description}{y}"]
    )
    
    df_close = df_close[left_cols]    
    df_close.columns = only_left.columns
    
    unique_columns = df_close.columns.tolist()
    unique_columns.remove('Description')
    
    df_close = df_close.drop_duplicates(unique_columns, ignore_index=True)

    df_far_left = (
        df.loc[(both & ~close), left_cols].drop_duplicates(ignore_index=True)
    )
    df_far_left.columns = only_left.columns

    df_far_right = (
        df.loc[(both & ~close), right_cols].drop_duplicates(ignore_index=True)
    )
    df_far_right.columns = only_right.columns

    merged_df = pd.concat(
        [df_left, df_right, df_close, df_far_left, df_far_right], ignore_index=True
    )    
    return merged_df.astype('string').drop_duplicates(ignore_index=True)

In [9]:
folder = Path.cwd().parent / "dados"

In [10]:
df_left = _filter_matlab(pd.read_parquet(folder / 'base.parquet.gzip')).drop_duplicates(ignore_index=True)

In [11]:
df_left.tail()

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
821179,89.1,-20.320200000000000,-40.322200000000000,"[MOS] FM-C4, A3, Radio E Televisao Espirito Sa...",230,1015028567,<NA>,256.0
821180,91.9,-29.7950194444443333,-50.0380000000000000,"[MOS] FM-C4, B1, Plus Radiodifusao Ltda (50409...",230,699673097,<NA>,256.0
821181,92.5,-21.457500000000000,-43.553800000000000,"[MOS] FM-C7, A1, Radio Serra Azul De Frequenci...",230,322376890,<NA>,256.0
821182,94.1,-10.0733333333333333,-67.0647222222221666,"[MOS] FM-C4, C, Radio, Tv E Jornal Impresso Am...",230,693210850,<NA>,256.0
821183,96.7,-4.448410000000000,-49.112800000000000,"[MOS] FM-C4, B1, Rede Norte De Comunicacao Ltd...",230,1007534009,<NA>,256.0


In [12]:
from extracao.updates import update_aero
df_right = pd.read_parquet(folder / 'aero.parquet.gzip').drop_duplicates(ignore_index=True)
# os.chdir(Path.cwd().parent / 'extracao/aero')

In [13]:
df_right.to_excel(folder / 'aero.xlsx', index=False)

In [14]:
df_right.tail()

,Frequency,Latitude,Longitude,Description
3340,995.0,-23.63138,-46.652061462402344,[AISG] DME - aeroporto de congonhas 34X
3341,995.0,-5.771808,-35.35019302368164,[AISG] DME - SÃO GONÇALO 34X
3342,995.0,-9.3369465,-54.95242691040039,[AISG] DME - CACHIMBO 34X
3343,995.0,-9.509805,-35.78694534301758,[AISG] DME - IMC 34X
3344,997.0,-15.878366,-47.906864166259766,[AISG] DME - BRASÍLIA 36X


In [15]:
df_left: pd.DataFrame  # DataFrame da esquerda a ser mesclado
df_right: pd.DataFrame  # DataFrame da direira a ser mesclado
on: str = "Frequency"  # Coluna usada como chave de mesclagem
coords: Tuple[str] = ("Latitude", "Longitude")
description: str = "Description"
suffixes: Tuple[str] = ("_x", "_y")  # Sufixo para as colunas que foram criadas

df: pd.DataFrame = pd.merge(
    df_left.astype("string"),
    df_right.astype("string"),
    on=on,
    how="outer",
    suffixes=suffixes,
    indicator=True,
    copy=False,
)

x, y = suffixes
lat, long = coords


left = df._merge == "left_only"
right = df._merge == "right_only"
both = df._merge == "both"
df = df.drop(columns=["_merge"])

left_cols: List[str] = [c for c in df.columns if y not in c]
right_cols: List[str] = [c for c in df.columns if x not in c]


only_left = df.loc[left, left_cols]
only_left.columns = [c.removesuffix(x) for c in left_cols]

only_right = df.loc[right, right_cols]
only_right.columns = [c.removesuffix(y) for c in right_cols]

both_columns = [f"{lat}{x}", f"{long}{x}", f"{lat}{y}", f"{long}{y}"]

df.loc[both, "Distance"] = df.loc[both, both_columns].apply(get_km_distance, axis=1)

close = df.loc[both, "Distance"] <= MAX_DIST

In [16]:
len(df_left) - len(only_left)

2533

In [17]:
len(df_right) - len(only_right)

2421

In [18]:
df_both = df[both].sort_values('Distance', ignore_index=True)

In [19]:
df_both

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,125.4,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
1,133.9,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
2,124.75,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
3,125.8,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
4,125.85,-31.390833,-54.109722,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699525020,A3E,6.0,-31.390833,-54.10972213745117,"[AISW] SBBG-COM, Rádio, Comandante Gustavo Kra...",0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...
58643,121.5,-7.595833333333333,-72.768611111111,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007719041,A3E,8.33,-3.8547223,-32.4283332824707,"[ICAO] EMERG 261/450, FERNANDO DE NORONHA",4485.739560
58644,131.225,-3.8633333333333333,-32.4230555555555,"[STEL] L, FA, Gol Linhas Aereas S.A. (50415208...",507,1011664469,A3E,6.0,-7.5961113,-72.76555633544922,"[ICAO] AOC U 100/100, CRUZEIRO DO SUL",4485.867377
58645,131.225,-7.596666666666667,-72.77444444444433,"[STEL] L, FA, Gol Linhas Aereas S.A. (50415208...",507,1011658825,A3E,6.0,-3.8547223,-32.4283332824707,"[ICAO] AOC U 100/100, FERNANDO DE NORONHA",4486.387171
58646,121.5,-3.847069444444333,-32.4014305555555,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007681141,A3E,8.33,-7.5961113,-72.76555633544922,"[ICAO] EMERG 261/450, CRUZEIRO DO SUL",4488.474873


In [21]:
both_left_cols = ['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x', 'Service', 'Station', 'Class', 'BW']
df_both_left = df_both.groupby(both_left_cols, as_index=False).first()
df_both_left

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,0.2,-19.415847,-42.548231,"[MOS] L, OP, Mge Transmissao S.A. (50410917141...",019,1001992978,J3D,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,391.489026
1,0.2,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,174.095832
2,0.2,-24.101667,-54.226667,"[MOS] L, OP, Companhia De Geracao Termica De E...",019,2525658,J3W,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,1247.860843
3,0.2,-5.198056,-49.016944,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,354635,A3E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,2098.546077
4,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
...,...,...,...,...,...,...,...,...,...,...,...,...
2528,136.825,-8.122802777777666,-34.919463888888835,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",0.539261
2529,329.6,-22.815361111111,-43.26133333333333,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242303,A3N,10.0,-22.839167,-43.239444732666016,"[DOC] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antôn...",3.463920
2530,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0,-22.796667,-43.2238883972168,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni...",2.928764
2531,332.9,-22.791916666666665,-43.221066666666665,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1009597296,A3N,10.0,-22.783333,-43.21666717529297,"[DOC] ILS/DME, RIO DE JANEIRO INTL. (ILS glide...",1.052450


In [22]:
both_right_cols = ['Frequency', 'Latitude_y', 'Longitude_y', 'Description_y']
df_both_right = df_both.groupby(both_right_cols, as_index=False).first()
df_both_right

,Frequency,Latitude_y,Longitude_y,Description_y,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Distance
0,0.2,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,174.095832
1,0.205,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,0.284589
2,0.205,-5.3861666,-35.53099822998047,[AISG] NDB - MAXARANGUAPE,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,2358.141702
3,0.225,-25.408667,-54.621665954589844,[AISG] NDB - ITAIPU,-21.718611,-41.338333,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,860026,J3E,1.0,1415.543991
4,0.23,-4.2493334,-69.93616485595703,[AISG] NDB - TABATINGA COVERAGE 50NM,-2.566667,-44.233333,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,355305,J8D,2.0,2862.064775
...,...,...,...,...,...,...,...,...,...,...,...,...
2416,332.0,2.8552778,-60.68361282348633,"[DOC] SBBV-NAV, ILS/DME 08 IBV, Atlas Brasil C...",-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0,3411.782441
2417,332.9,-22.783333,-43.21666717529297,"[DOC] ILS/DME, RIO DE JANEIRO INTL. (ILS glide...",-22.791916666666665,-43.221066666666665,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1009597296,A3N,10.0,1.052450
2418,332.9,-22.804445,-43.26416778564453,"[DOC] SBGL-NAV, ILS/DME 28 ILM, Galeão - Antôn...",-22.791916666666665,-43.221066666666665,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1009597296,A3N,10.0,4.637764
2419,332.9,-23.433332,-46.45000076293945,"[DOC] ILS, SAO PAULO GUARULHOS INTL. (ILS glid...",-22.791916666666665,-43.221066666666665,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1009597296,A3N,10.0,338.295591


In [48]:
ents_both_left =  set(df_both_left[both_left_cols].apply(tuple, 1).tolist())
ents_both_right = set(df_both_right[both_right_cols].apply(tuple, 1).tolist())

In [39]:
df_merge = df_both_right[df_both_right.Distance <= MAX_DIST].drop('Distance', axis=1).copy()
df_merge = df_merge.groupby(both_left_cols, as_index=False).first()
df_merge

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y
0,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...
1,0.28,-28.280791666666666,-54.170222222222165,"[MOS] L, RC, Paula Gabriela Dalla Porta (50413...",108,1002970897,A2A,2.0,-28.27931,-54.17194366455078,[AISG] NDB - SANTO ÂNGELO OPR RIO SUL.
2,0.28,-28.678911111111,-49.06675833333333,"[MOS] L, RC, Secretaria De Estado Da Infra-Est...",108,1000481252,A2A,2.0,-28.67885,-49.066715240478516,[AISG] NDB - JAGUARUNA COVERAGE 30NM. Other HR...
3,1003.0,-22.83,-43.23694444444433,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242109,M1A,700.0,-22.829702,-43.23648452758789,[AISG] DME - GALEÃO 42X
4,109.1,-21.787222222222166,-43.386111111111,"[MOS] L, AL, Sinart - Sociedade Nacional De Ap...",108,696597080,A3E,6.0,-21.783333,-43.38333511352539,"[ICAO] ILS/DME, JUIZ DE FORA"
...,...,...,...,...,...,...,...,...,...,...,...
1199,136.825,-29.990277777777667,-51.18138888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009266109,A3E,6.0,-29.994444,-51.17166519165039,"[ICAO] AOC U 100/100, PORTO ALEGRE"
1200,136.825,-8.122802777777666,-34.919463888888835,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE"
1201,329.6,-22.815361111111,-43.26133333333333,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242303,A3N,10.0,-22.839167,-43.239444732666016,"[DOC] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antôn..."
1202,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0,-22.796667,-43.2238883972168,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni..."


In [40]:
right_cols

['Frequency',
 'Service',
 'Station',
 'Class',
 'BW',
 'Latitude_y',
 'Longitude_y',
 'Description_y']

In [49]:
drop_left = set(df_merge[left_cols].apply(tuple,1))
drop_right = set(df_merge[right_cols].apply(tuple,1))

In [51]:
len(ents_both_left.difference(drop_left))

1329

In [52]:
len(ents_both_right.difference(drop_right))

1937

In [45]:
df_both = df_both[~df_both[left_cols].apply(tuple,1).isin(drop_left)]
df_both = df_both[~df_both[right_cols].apply(tuple,1).isin(drop_right)]
df_both


,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
16,131.925,-10.985277777777666,-37.07333333333333,"[STEL] L, FA, Omni Taxi Aereo S/A (50011626100...",507,1005818476,A3E,6.0,-10.985278,-37.073333740234375,"[ICAO] AOC U 100/100, ARACAJU",0.000051
17,131.925,-10.985277777777666,-37.07333333333333,"[STEL] L, FA, Omni Taxi Aereo S/A (50011626100...",507,1005818468,A3E,6.0,-10.985278,-37.073333740234375,"[ICAO] AOC U 100/100, ARACAJU",0.000051
19,130.95,-22.9705555555555,-43.21638888888884,"[STEL] L, FA, Helisul Taxi Aereo Ltda (5041602...",507,1006947318,A3E,5.6,-22.970556,-43.21638870239258,"[ICAO] AOC U 100/100, LAGOA",0.000053
50,131.25,-4.566111111111,-37.79833333333333,"[STEL] L, FA, Departamento Estadual De Rodovia...",507,1003961387,A3E,5.6,-4.566111,-37.79833221435547,"[ICAO] AOC U 100/100, ARACATI | [AISW] SBAC-CO...",0.000125
80,125.5,-4.242222,-56.000833,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699506557,A3E,6.8,-4.2422223,-56.000831604003906,"[ICAO] TWR 25/40, ITAITUBA | [AISW] SBIH-COM, ...",0.000158
...,...,...,...,...,...,...,...,...,...,...,...,...
58577,131.875,2.841111111111,-60.69222222222216,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1014126786,A3E,6.0,-31.71889,-52.32805633544922,"[ICAO] AOC U 100/100, PELOTAS",3925.926973
58588,121.5,-22.113111111111,-41.175361111111,"[STEL] L, FA, Petroleo Brasileiro S A Petrobra...",507,1012613264,A3E,8.33,0.60694444,-69.20138549804688,"[ICAO] EMERG 261/450, IAUARETÊ",3945.955665
58589,121.5,-22.113111111111,-41.175361111111,"[STEL] L, FA, Petroleo Brasileiro S A Petrobra...",507,1012613299,A3E,8.33,0.60694444,-69.20138549804688,"[ICAO] EMERG 261/450, IAUARETÊ",3945.955665
58615,121.5,-5.763944,-35.368297,"[MOS] L, FA, Inframerica Concessionaria Do Aer...",019,699940320,A3E,8.33,-7.5961113,-72.76555633544922,"[ICAO] EMERG 261/450, CRUZEIRO DO SUL",4138.711282


In [56]:
df_both_left = df_both.groupby(both_left_cols, as_index=False).first()
df_both_left

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,0.2,-19.415847,-42.548231,"[MOS] L, OP, Mge Transmissao S.A. (50410917141...",019,1001992978,J3D,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,391.489026
1,0.2,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,174.095832
2,0.2,-24.101667,-54.226667,"[MOS] L, OP, Companhia De Geracao Termica De E...",019,2525658,J3W,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,1247.860843
3,0.2,-5.198056,-49.016944,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,354635,A3E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,2098.546077
4,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
...,...,...,...,...,...,...,...,...,...,...,...,...
1324,136.825,-22.9125,-43.16638888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1005172525,A3E,6.0,-19.373888,-43.58277893066406,"[ICAO] AOC U 100/100, CONFINS",394.171092
1325,136.825,-4.570427777777667,-37.80143888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1008773171,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",506.015045
1326,136.825,-5.062686111111,-42.82136388888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009303802,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",936.787233
1327,136.825,-7.215,-39.27194444444434,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009307174,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",490.307930


In [57]:
df_both_right = df_both.groupby(both_right_cols, as_index=False).first()
df_both_right

,Frequency,Latitude_y,Longitude_y,Description_y,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Distance
0,0.2,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,174.095832
1,0.205,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,0.284589
2,0.205,-5.3861666,-35.53099822998047,[AISG] NDB - MAXARANGUAPE,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,2358.141702
3,0.225,-25.408667,-54.621665954589844,[AISG] NDB - ITAIPU,-21.718611,-41.338333,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,860026,J3E,1.0,1415.543991
4,0.23,-4.2493334,-69.93616485595703,[AISG] NDB - TABATINGA COVERAGE 50NM,-2.566667,-44.233333,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,355305,J8D,2.0,2862.064775
...,...,...,...,...,...,...,...,...,...,...,...,...
1932,136.825,-19.373888,-43.58277893066406,"[ICAO] AOC U 100/100, CONFINS",-19.635833333333334,-43.961944444444335,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009307719,A3E,6.0,49.243739
1933,136.825,-23.006945,-47.13444519042969,"[ICAO] AOC U 100/100, CAMPINAS",-22.9125,-43.16638888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1005172525,A3E,6.0,407.057740
1934,136.825,-29.994444,-51.17166519165039,"[ICAO] AOC U 100/100, PORTO ALEGRE",-20.457222222222168,-54.66888888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009266141,A3E,6.0,1113.516782
1935,136.825,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",-7.215,-39.27194444444434,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009307174,A3E,6.0,490.307930


In [60]:
df_both_left[df_both_left.Distance <= MAX_DIST].groupby(both_left_cols, as_index=False).first()

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
1,118.0,-10.294283,-48.359475,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699506786,A3E,6.0,-10.29,-48.3577766418457,"[AISW] SBPJ-COM, Torre, Brigadeiro Lysias Rodr...",0.508961
2,118.0,-19.849167,-43.958611,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699525276,A3E,6.0,-19.851944,-43.95055389404297,"[ICAO] TWR 25/40, BELO HORIZONTE | [AISW] SBBH...",0.898193
3,118.0,-21.135833,-47.776667,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526248,A3E,6.0,-21.136389,-47.776668548583984,"[AISW] SBRP-COM, Torre, Leite Lopes",0.061560
4,118.0,-22.806688888888832,-43.232969444444336,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007543024,A3E,8.33,-22.81,-43.25055694580078,"[ICAO] TWR 25/40, RIO DE JANEIRO - GALEÃO | [A...",1.842506
...,...,...,...,...,...,...,...,...,...,...,...,...
261,136.125,-21.763333,-48.400833,"[MOS] L, FA, Embraer S.A. (50406881138, 692616...",019,692616217,A3E,6.0,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",0.420848
262,136.125,-21.763333333333332,-48.40083333333333,"[STEL] L, FA, Embraer S.A. (50419060766, 10109...",507,1010931986,A3E,6.0,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",0.420805
263,136.125,-21.763333333333332,-48.40083333333333,"[STEL] L, FA, Embraer S.A. (50419060766, 10109...",507,1010932010,A3E,6.0,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",0.420805
264,136.125,-21.763333333333332,-48.40083333333333,"[STEL] L, FA, Embraer S.A. (50419060766, 10109...",507,1010932036,A3E,6.0,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",0.420805


In [61]:
df_both_right[df_both_right.Distance <= MAX_DIST].groupby(both_right_cols, as_index=False).first()

,Frequency,Latitude_y,Longitude_y,Description_y,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Distance
0,0.205,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,0.284589
1,118.0,-10.29,-48.3577766418457,"[AISW] SBPJ-COM, Torre, Brigadeiro Lysias Rodr...",-10.294283,-48.359475,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699506786,A3E,6.0,0.508961
2,118.0,-10.29,-48.3577766418457,"[ICAO] TWR 25/40, PALMAS | [AISW] SBPJ-COM, To...",-10.294283,-48.359475,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699506786,A3E,6.0,0.508961
3,118.0,-19.851944,-43.95055389404297,"[AISW] SBBH-COM, Torre, Pampulha - Carlos Drum...",-19.849167,-43.958611,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699525276,A3E,6.0,0.898193
4,118.0,-19.851944,-43.95055389404297,"[ICAO] TWR 25/40, BELO HORIZONTE | [AISW] SBBH...",-19.849167,-43.958611,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699525276,A3E,6.0,0.898193
...,...,...,...,...,...,...,...,...,...,...,...,...
185,135.2,-23.435556,-46.47305679321289,"[AISW] SBGR-COM, Torre, Guarulhos - Governador...",-23.425556,-46.477778,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526302,A3E,6.0,1.208034
186,135.2,-23.435556,-46.47305679321289,"[ICAO] TWR 25/40, GUARULHOS | [AISW] SBGR-COM,...",-23.425556,-46.477778,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699526302,A3E,6.0,1.208034
187,136.05,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",-21.761666666666667,-48.4080555555555,"[STEL] L, FA, Embraer S.A. (50419060766, 10117...",507,1011702310,A3E,6.0,0.461852
188,136.125,-21.764444,-48.40472412109375,"[ICAO] AOC U 100/100, GAVIÃO PEIXOTO",-21.763333333333332,-48.40083333333333,"[STEL] L, FA, Embraer S.A. (50419060766, 10109...",507,1010932036,A3E,6.0,0.420805


In [38]:
df_merge["Description_x"] = (
        df_merge['Description_x'] + " | " + df_merge['Description_y']
    )
    
df_merge = df_merge[left_cols]    
df_merge.columns = only_left.columns
df_merge

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7
1,0.28,-28.280791666666666,-54.170222222222165,"[MOS] L, RC, Paula Gabriela Dalla Porta (50413...",108,1002970897,A2A,2.0
2,0.28,-28.678911111111,-49.06675833333333,"[MOS] L, RC, Secretaria De Estado Da Infra-Est...",108,1000481252,A2A,2.0
3,1003.0,-22.83,-43.23694444444433,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242109,M1A,700.0
4,109.1,-21.787222222222166,-43.386111111111,"[MOS] L, AL, Sinart - Sociedade Nacional De Ap...",108,696597080,A3E,6.0
...,...,...,...,...,...,...,...,...
1199,136.825,-29.990277777777667,-51.18138888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009266109,A3E,6.0
1200,136.825,-8.122802777777666,-34.919463888888835,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0
1201,329.6,-22.815361111111,-43.26133333333333,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242303,A3N,10.0
1202,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0


In [34]:
merged = pd.DataFrame(columns=df_left.columns)
for _, row in df_merge.iterrows():
    record = tuple(row[c] for c in both_right_cols)
    


True


In [30]:
row.

Pandas(Index=1, Frequency='0.205', Latitude_y='-1.4863334', Longitude_y='-56.39783477783203', Description_y='[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MINERAÇÃO    RIO NORTE S/A', Latitude_x='-1.4883333333333333', Longitude_x='-56.399444444444335', Description_x='[MOS] L, RC, Mineracao Rio Do Norte S.A. (50014083426, 684424649), Oriximiná/PA', Service='108', Station='684424649', Class='A2A', BW='2.7')

In [22]:
df_close = df_close.groupby(['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x', 'Service', 'Station', 'Class', 'BW'], as_index=False).first()
df_close

,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
1,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,N0N,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
2,0.28,-28.280791666666666,-54.170222222222165,"[MOS] L, RC, Paula Gabriela Dalla Porta (50413...",108,1002970897,A2A,2.0,-28.27931,-54.17194366455078,[AISG] NDB - SANTO ÂNGELO OPR RIO SUL.,0.235553
3,0.28,-28.678911111111,-49.06675833333333,"[MOS] L, RC, Secretaria De Estado Da Infra-Est...",108,1000481252,A2A,2.0,-28.67885,-49.066715240478516,[AISG] NDB - JAGUARUNA COVERAGE 30NM. Other HR...,0.007976
4,1003.0,-22.83,-43.23694444444433,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242109,M1A,700.0,-22.839167,-43.239444732666016,"[DOC] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antôn...",1.047109
...,...,...,...,...,...,...,...,...,...,...,...,...
1465,136.825,-29.990277777777667,-51.18138888888883,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009266109,A3E,6.0,-29.994444,-51.17166519165039,"[ICAO] AOC U 100/100, PORTO ALEGRE",1.045778
1466,136.825,-8.122802777777666,-34.919463888888835,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",0.539261
1467,329.6,-22.815361111111,-43.26133333333333,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242303,A3N,10.0,-22.839167,-43.239444732666016,"[DOC] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antôn...",3.463920
1468,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0,-22.796667,-43.2238883972168,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni...",2.928764


In [27]:
entitites = set(df_close[['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x']].apply(tuple, 1).to_list())
entitites

{('121.7',
  '-22.910556',
  '-43.166667',
  '[MOS] L, FA, Empresa Brasileira De Infra-Estrutura Aeroportuaria (11030016470, 699526256+3), Rio de Janeiro/RJ'),
 ('132.75',
  '-23.438333',
  '-46.469722',
  '[MOS] L, FA, Empresa Brasileira De Infra-Estrutura Aeroportuaria (11030016470, 699471427), Guarulhos/SP'),
 ('119.65',
  '-20.465',
  '-54.6850555555555',
  '[STEL] L, FA, Comando Da Aeronautica (50415723132, 1007684906), Campo Grande/MS'),
 ('130.5',
  '-5.530278',
  '-47.457778',
  '[MOS] L, FA, Tam Linhas Aereas S/A (02030398152, 627454178), Imperatriz/MA'),
 ('131.925',
  '-22.338388888888833',
  '-41.76058333333334',
  '[STEL] L, FA, Omni Taxi Aereo S/A (50011626100, 1005581280), Macaé/RJ'),
 ('121.5',
  '-11.618888888888833',
  '-50.6925',
  '[STEL] L, FA, Comando Da Aeronautica (50415723132, 1007719270), São Félix do Araguaia/MT'),
 ('123.7',
  '-22.636444444444333',
  '-50.45133333333333',
  '[STEL] L, FA, Comando Da Aeronautica (50415723132, 1007691937), Assis/SP'),
 ('125.

In [ ]:
# df_close[f"{description}{x}"] = (
#     df_close[f"{description}{x}"] + " | " + df_close[f"{description}{y}"]
# )

# df_close = df_close[left_cols]    
# df_close.columns = only_left.columns

# unique_columns = df_close.columns.tolist()
# unique_columns.remove('Description')

# df_close = df_close.drop_duplicates(unique_columns, ignore_index=True)

In [ ]:
df[both] = df[both].sort_values(by='Distance')
df[both]

C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_2352\829667317.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[both] = df[both].sort_values(by='Distance')
C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_2352\829667317.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[both]


,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
28758,125.8,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
28228,124.75,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
65645,133.9,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
28470,125.4,-0.9763888888888334,-62.92222222222217,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007724614,A3E,8.33,-0.9763889,-62.92222213745117,"[ICAO] ACC-U C-261/450, BARCELOS",0.000010
28964,125.85,-31.390833,-54.109722,"[MOS] L, FA, Empresa Brasileira De Infra-Estru...",019,699525020,A3E,6.0,-31.390833,-54.10972213745117,"[AISW] SBBG-COM, Rádio, Comandante Gustavo Kra...",0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...
24727,121.5,-7.595833333333333,-72.768611111111,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007719041,A3E,8.33,-3.8547223,-32.4283332824707,"[ICAO] EMERG 261/450, FERNANDO DE NORONHA",4485.739560
51029,131.225,-3.8633333333333333,-32.4230555555555,"[STEL] L, FA, Gol Linhas Aereas S.A. (50415208...",507,1011664469,A3E,6.0,-7.5961113,-72.76555633544922,"[ICAO] AOC U 100/100, CRUZEIRO DO SUL",4485.867377
51511,131.225,-7.596666666666667,-72.77444444444433,"[STEL] L, FA, Gol Linhas Aereas S.A. (50415208...",507,1011658825,A3E,6.0,-3.8547223,-32.4283332824707,"[ICAO] AOC U 100/100, FERNANDO DE NORONHA",4486.387171
22413,121.5,-3.847069444444333,-32.4014305555555,"[STEL] L, FA, Comando Da Aeronautica (50415723...",507,1007681141,A3E,8.33,-7.5961113,-72.76555633544922,"[ICAO] EMERG 261/450, CRUZEIRO DO SUL",4488.474873


In [ ]:
df[both].groupby(['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x', 'Service', 'Station', 'Class', 'BW'], as_index=False).first()

C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_2352\2716649523.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[both].groupby(['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x', 'Service', 'Station', 'Class', 'BW'], as_index=False).first()


,Frequency,Latitude_x,Longitude_x,Description_x,Service,Station,Class,BW,Latitude_y,Longitude_y,Description_y,Distance
0,0.2,-19.415847,-42.548231,"[MOS] L, OP, Mge Transmissao S.A. (50410917141...",019,1001992978,J3D,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,391.489026
1,0.2,-22.916111,-43.768611,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557637,J9E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,174.095832
2,0.2,-24.101667,-54.226667,"[MOS] L, OP, Companhia De Geracao Termica De E...",019,2525658,J3W,4.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,1247.860843
3,0.2,-5.198056,-49.016944,"[MOS] L, OP, Centrais Eletricas Do Norte Do Br...",019,354635,A3E,8.0,-22.923334,-42.07149887084961,[AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...,2098.546077
4,0.205,-1.4883333333333333,-56.399444444444335,"[MOS] L, RC, Mineracao Rio Do Norte S.A. (5001...",108,684424649,A2A,2.7,-1.4863334,-56.39783477783203,[AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...,0.284589
...,...,...,...,...,...,...,...,...,...,...,...,...
2528,136.825,-8.122802777777666,-34.919463888888835,"[STEL] L, FA, Azul Linhas Aereas Brasileiras S...",507,1009265412,A3E,6.0,-8.126389,-34.92277908325195,"[ICAO] AOC U 100/100, RECIFE",0.539261
2529,329.6,-22.815361111111,-43.26133333333333,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242303,A3N,10.0,-22.839167,-43.239444732666016,"[DOC] SBGL-NAV, ILS/DME 15 IJB, Galeão - Antôn...",3.463920
2530,332.0,-22.799722222222165,-43.252222222222166,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1008242095,A3N,10.0,-22.796667,-43.2238883972168,"[DOC] SBGL-NAV, ILS 10 CAT II, Galeão - Antôni...",2.928764
2531,332.9,-22.791916666666665,-43.221066666666665,"[MOS] L, RC, Comando Da Aeronautica (504177201...",108,1009597296,A3N,10.0,-22.783333,-43.21666717529297,"[DOC] ILS/DME, RIO DE JANEIRO INTL. (ILS glide...",1.052450


In [ ]:
df_far_left = (
    df.loc[(both & ~close), left_cols].drop_duplicates(ignore_index=True)
)
df_far_left.columns = only_left.columns

df_far_right = (
    df.loc[(both & ~close), right_cols].drop_duplicates(ignore_index=True)
)
df_far_right.columns = only_right.columns

merged_df = pd.concat(
    [df_left, df_right, df_close, df_far_left, df_far_right], ignore_index=True
)    

In [ ]:
df

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,0.028,-22.662778,-43.476389,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557670,J9E,8.0
1,0.03,-18.41,-49.1,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859966,J3E,1.0
2,0.03,-22.774167,-47.004444,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859753,J3E,1.0
3,0.03,-22.926667,-43.265,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,859761,J3E,0.5
4,0.03,-23.441667,-46.590833,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",019,1557823,J3E,1.0
...,...,...,...,...,...,...,...,...
880852,0.31,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,108,1002356390,A2A,2.1
880853,0.31,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,108,1002356390,A2A,2.04
880854,0.31,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,108,1002956207,A2A,2.04
880855,0.31,-16.703382,-43.81910705566406,[AISG] NDB - MONTES CLAROS COVERAGE 50NM OPR N...,108,1002956223,A2A,2.04


In [ ]:
df_left[df_left.Station == '1002356390']

,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
820991,0.31,-23.995100000000000,-46.249000000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002356390,A2A,2.1
820992,0.31,-23.995100000000000,-46.249000000000000,"[MOS] L, RC, Sapura Navegacao Maritima S.A. (5...",108,1002356390,A2A,2.04
